# 基于AgentRuntime服务化组件

## 目标
使用 AgentRuntime 对组件进行服务化。

AgentRuntime 是对组件（Component）的服务化封装，具体有如下几个功能：
- 一键服务化组件: 使得组件能够以服务的形式运行，支持 API 调用和对话框交互。
- Session 数据管理: 提供 Session 数据的管理功能，允许跟踪和存储用户会话数据。
- 请求时鉴权: 支持在请求时进行认证，确保安全性。


## 准备工作
### 安装Python SDK

appbuilder 支持使用 pip 安装（要求Python >= 3.8），并且 AgentRuntime 服务化组件依赖 `appbuilder-sdk[serve]`

In [ ]:
pip install appbuilder-sdk 'appbuilder-sdk[serve]'

## 基本用法

### 快速开始

下面的示例会基于 Playground 组件，在 8091 端口部署 Web 服务: 

In [ ]:
import os
import appbuilder

# 使用组件之前，请前往千帆AppBuilder官网创建密钥，流程详见：https://cloud.baidu.com/doc/AppBuilder/s/Olq6grrt6#1、创建密钥
os.environ["APPBUILDER_TOKEN"] = '...'

component = appbuilder.Playground(
    prompt_template="{query}",
    model="ERNIE-Bot"
)

agent = appbuilder.AgentRuntime(component=component)
agent.serve(port=8091)

通过 Shell 命令测试启动的服务, 请求 Body 为组件 run 方法的入参：


In [ ]:
curl --location 'http://0.0.0.0:8091/chat' \
--header 'Content-Type: application/json' \
--data '{
    "message": "海淀区的面积是多少",
    "stream": false
}'

## AgentRuntime 参数说明

### 1. 类初始化参数说明

AgentRuntime 初始化接受两个参数。

| 参数名称 | 参数类型 | 是否必须 | 描述 | 示例值 |
|--|--|--|--|--|
| component | Component | 是 | 可运行的 Component, 该 Component 需要实现 run(message, stream, **args) 方法。 | Playground(prompt_template="{query}", model="ERNIE-Bot") |
| user_session_config | sqlalchemy.engine.URL\|Str\|None | 否 | 会话 Session 数据存储的数据库配置，遵循 sqlalchemy 后端定义，可参考[文档](https://docs.sqlalchemy.org/en/20/core/engines.html#backend-specific-urls)。默认使用 sqlite:///user_session.db，即本地的 SQLite 存储 | "sqlite:///user_session.db" |

### 2. API 服务参数

#### 2.1 请求参数

**接口定义**

| URL | Method |
|--|--|
| /chat | POST |

**Header 参数**

| 参数名称 | 是否必须 | 描述 | 示例值 |
|--|--|--|--|
| Content-Type | 是 | 必须设置为"application/json" | "application/json" |
| X-Appbuilder-Token | 否 | 开启请求时认证能力时需要带入 APPBUILDER_TOKEN 进行鉴权 | 前往千帆AppBuilder官网创建密钥，流程详见[文档](https://cloud.baidu.com/doc/AppBuilder/s/Olq6grrt6#1、创建密钥) |

**Body 参数**

| 参数名称 | 参数类型 | 是否必须 | 描述 | 示例值 |
|--|--|--|--|--|
| message | Any | 是 | 透传到 component 的 run 方法的 message 参数 | "海淀区的面积是多少" |
| stream | Bool | 否 | 是否流式调用。透传到 component 的 run 方法的 stream 参数。默认为 false | false |
| session_id | Str | 否 | 用于标示同一个会话（Session）。如果不传该值，后端会自动生成 session_id，在响应参数中返回 | "99680089-5acb-4298-9ade-a1a3f6c28102" |
| 其他参数 | Any | 否 | 透传到 component 的 run 方法 | - |


#### 2.2 响应参数
分为非流式响应和流式响应。

**非流式响应**

| 参数名称 | 参数类型 | 描述 | 示例值 |
|--|--|--|--|
| code | Int | 错误码。值为0表示成功，否则为失败。非0错误详见错误码部分描述 | 0 |
| message | Str | 错误信息描述。 | "Missing input variable query in message ['海淀区的面积是多少']" |
| result | Object | 请求结果 | - |
| + answer_message | Object | 组件返回值，由返回的 Message 序列化得到 | {"content":"海淀区是北京市的一个区，位于北京市主城区西部和西北部，东与西城区、朝阳区相邻，南与丰台区毗连，西与石景山区、门头沟区交界，北与昌平区接壤。海淀区的面积为**431平方千米**，约占北京市总面积的2.6%。","extra":{},"id":"6b4e5019-a708-4bc5-a6ec-595fb4285677","mtype":"dict","name":"msg"} |
| + session_id | Str | 用于标示同一个会话（Session） | "99680089-5acb-4298-9ade-a1a3f6c28102" |

**流式响应**

流式数据以追加的形式返回。流式和非流式的数据结构一致，不再描述。

#### 2.3 响应示例

分为非流式响应和流式响应。

**非流式响应**

```shell
{
  "code": 0,
  "message": "",
  "result": {
    "answer_message": {
      "content": "海淀区是北京市的一个区，位于北京市主城区西部和西北部，东与西城区、朝阳区相邻，南与丰台区毗连，西与石景山区、门头沟区交界，北与昌平区接壤。海淀区的面积为**431平方千米**，约占北京市总面积的2.6%。",
      "extra": {},
      "id": "6b4e5019-a708-4bc5-a6ec-595fb4285677",
      "mtype": "dict",
      "name": "msg"
    },
    "session_id": "99680089-5acb-4298-9ade-a1a3f6c28102"
  }
}
```

**流式响应**

```shell
data: {"code": 0, "message": "", "result": {"session_id": "663303a9-d83d-481f-a084-872ece87989c", "answer_message": {"content": "海淀区", "extra": {}}}}

data: {"code": 0, "message": "", "result": {"session_id": "663303a9-d83d-481f-a084-872ece87989c", "answer_message": {"content": "，隶属于北京市，位于北京市主城区西部和西北部，东与西城区、朝阳区相邻，南与丰台区毗连，", "extra": {}}}}

data: {"code": 0, "message": "", "result": {"session_id": "663303a9-d83d-481f-a084-872ece87989c", "answer_message": {"content": "西与石景山区、门头沟区交界，北与昌平区接壤，总面积**431平方千米**。", "extra": {}}}}

data: {"code": 0, "message": "", "result": {"session_id": "663303a9-d83d-481f-a084-872ece87989c", "answer_message": {"content": "", "extra": {}}}}
```

#### 2.4 错误码
| 错误码 | 描述 |
|--|--|
| 400 | 客户端请求参数错误 |
| 1000 | 服务端执行错误 |

## 高级用法

### 1. 一键服务化组件 
AgentRuntime 可以快速组件以服务的形式运行，支持 API 调用和对话框交互。

**1.1 API调用**

API 调用的基础用法在快速开始小结已经给出，这里不再赘述。

下面介绍使用 `gunicorn` 启动生产级 Web 服务的方法，`gunicorn` 是一个适用于 UNIX 的 Python WSGI HTTP 服务器，详见[项目链接](https://github.com/benoitc/gunicorn)。

首先创建 `app.py` 文件，暴露 Flask App：

In [ ]:
import os
import appbuilder

# 使用组件之前，请前往千帆AppBuilder官网创建密钥，流程详见：https://cloud.baidu.com/doc/AppBuilder/s/Olq6grrt6#1、创建密钥
os.environ["APPBUILDER_TOKEN"] = '...'

def get_flask_app():
    component = appbuilder.Playground(
        prompt_template="{query}",
        model="ERNIE-Bot"
    )
    agent = appbuilder.AgentRuntime(component=component)
    return agent.create_flask_app()

基于 `gunicorn` 启动生产级服务：

In [ ]:
# 服务工作进程数
SERVER_WORKER_AMOUNT=8
# 服务工作进程启动方式
SERVER_WORKER_CLASS=gevent
# 服务超时时间
GUNICORN_TIMEOUT=60

gunicorn \
  --bind "0.0.0.0:8091" \
  --workers ${SERVER_WORKER_AMOUNT} \
  --worker-class ${SERVER_WORKER_CLASS} \
  --timeout ${GUNICORN_TIMEOUT} \
  "app:get_flask_app()"

**1.2 对话框交互**

基于 chainlit 的对话框交互对被服务化的组件的 message 参数更加严格，要求能够接受 Str 的基础类型。

执行下面的代码，会启动一个 chainlit 页面，页面地址：0.0.0.0:8091

In [ ]:
import os
import appbuilder

# 使用组件之前，请前往千帆AppBuilder官网创建密钥，流程详见：https://cloud.baidu.com/doc/AppBuilder/s/Olq6grrt6#1、创建密钥
os.environ["APPBUILDER_TOKEN"] = '...'

component = appbuilder.Playground(
    prompt_template="{query}",
    model="ERNIE-Bot"
)

agent = appbuilder.AgentRuntime(component=component)
agent.chainlit_demo(port=8091)

Chainlit Demo页面示意图如下所示，

![chainlit demo](image/agent_runtime_with_chainlit_demo.png)

**1.3 对话框交互AppBuilderClient**

基于 chainlit 的对话框交互AppBuilderClient，可实现对话交互。

执行下面的代码，会启动一个 chainlit 页面，页面地址：0.0.0.0:8091。可在页面上上传文档（可选）、执行对话。

In [ ]:
import os
import appbuilder

# 使用组件之前，请前往千帆AppBuilder官网创建密钥，流程详见：https://cloud.baidu.com/doc/AppBuilder/s/Olq6grrt6#1、创建密钥
os.environ["APPBUILDER_TOKEN"] = "..."
# 使用之前，在官网个人空间获取应用ID，如下图
app_id= "..."
agent_builder = appbuilder.AppBuilderClient(app_id)
agent = appbuilder.AgentRuntime(component=agent_builder)
agent.chainlit_agent(port=8091)

在官网个人空间获取应用ID
![get app_id](../app_builder_resources/app_id.png)

使用服务上传文件并对话示意图如下所示
![chainlit demo](./image/agent_runtime_with_chainlit_agent.png)

### 2. Session 数据管理
AgentRuntime 提供 Session 数据的管理功能，允许跟踪和存储用户会话数据。一般只有在二次开发的组件需要使用该能力。

**2.1 二次开发组件**

二次开发的组件需要重写组件的 run(message, stream, **args)方法，并且至少需要有 message 和 stream 两个参数。

下面基于 QueryRewrite 和 Playground 两个组件，开发 PlaygroundWithHistory 组件，该组件需要对会话数据进行操作。

当使用 Component 独立运行时，会话数据会被存储于内存。


In [ ]:
import os
import logging
from appbuilder.core.component import Component
from appbuilder import (
    AgentRuntime, UserSession, Message, QueryRewrite, Playground,
)

# 使用组件之前，请前往千帆AppBuilder官网创建密钥，流程详见：https://cloud.baidu.com/doc/AppBuilder/s/Olq6grrt6#1、创建密钥
os.environ["APPBUILDER_TOKEN"] = '...'

class PlaygroundWithHistory(Component):
    def __init__(self):
        super().__init__()
        self.query_rewrite = QueryRewrite(model="ERNIE Speed-AppBuilder")
        self.playground = Playground(
            prompt_template="{query}",
            model="ERNIE-Bot"
        )

    def run(self, message: Message, stream: bool=False):
        user_session = UserSession()
        # 获取 Session 历史数据
        history_queries = user_session.get_history("query", limit=1)
        history_answers = user_session.get_history("answer", limit=1)

        # query 改写
        if history_queries and history_answers:
            history = []
            for query, answer in zip(history_queries, history_answers):
                history.extend([query.content, answer.content])
            logging.info(f"history: {history}")
            message = self.query_rewrite(
                Message(history + [message.content]), rewrite_type="带机器人回复")
        logging.info(f"message: {message}") 

        # 执行 playground
        answer = self.playground.run(message, stream)

        # 保存本轮数据
        user_session.append({
            "query": message,
            "answer": answer,
        }) 
        return answer

# component 可以独立运行，session数据会被保存于内存
playground_with_history_component = PlaygroundWithHistory()
print(playground_with_history_component.run(Message("海淀区的面积是多少"), stream=False))

**2.2 会话数据存储数据库**

使用 AgentRuntime 对 Component 服务化，会话数据会被存储于数据库。
下面的代码以 SQLite 为例展示该能力，更多数据库配置详见[文档](https://docs.sqlalchemy.org/en/20/core/engines.html#backend-specific-urls)

In [ ]:
user_session_config = "sqlite:///foo.db"
agent = appbuilder.AgentRuntime(
    component=playground_with_history_component, 
    user_session_config=user_session_config)
agent.serve(port=8091)

### 3. 请求时鉴权
AgentRuntime 支持在请求时进行认证，确保安全性。

使用该能力，在初始化组件时需要设置 lazy 鉴权：

In [ ]:
import appbuilder

# 无需配置 APPBUILDER_TOKEN 环境变量

component = appbuilder.Playground(
    prompt_template="{query}",
    model="ERNIE-Bot",
    lazy_certification=True, # 设置 lazy 鉴权，在创建时不进行认证
)

agent = appbuilder.AgentRuntime(component=component)
agent.serve(port=8091)

当初始化组件时进行了 lazy 鉴权，请求时请求头必须带上 `X-Appbuilder-Token` （即Appbuilder密钥，获取流程详见[千帆AppBuilder官网创建密钥](https://cloud.baidu.com/doc/AppBuilder/s/Olq6grrt6#1、创建密钥)）进行鉴权：

In [ ]:
curl --location 'http://0.0.0.0:8091/chat' \
    --header 'Content-Type: application/json' \
    --header 'X-Appbuilder-Token: ...' \
    --data '{
        "message": "海淀区的面积是多少",
        "stream": false
    }'

### 4. 查看user_session.db储存信息


使用该能力，查看用户对话信息。

In [83]:
import sqlite3  
  
# 连接到 SQLite 数据库  
# 如果文件不存在，会自动在当前目录创建:  
user_session_path = '/Users/yinjiaqi/workspace/user_session.db' 
conn = sqlite3.connect(user_session_path)  
cursor = conn.cursor()  

执行 SQL 语句，列出SQLite数据库中的所有表

In [85]:
# 执行一条 SQL 语句，列出所有表  
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")  
print(cursor.fetchall())  

[('appbuilder_session_messages',)]


查询appbuilder_session_messages表的列信息,appbuilder_session_messages表的列信息为:
- id
- session_id
- request_id
- message_key
- message_value
- created_at
- updated_at
- deleted

In [79]:
cursor.execute("PRAGMA table_info(appbuilder_session_messages);")  
columns_info = cursor.fetchall()  

column_names = [info[1] for info in columns_info]  # info[1]是列名的位置  
for column_name in column_names:  
    print(column_name)   

id
session_id
request_id
message_key
message_value
created_at
updated_at
deleted


查询表中的特定数据【以message_value信息为例】
输出content、id、token_usage等信息

In [80]:
import json
cursor.execute("SELECT message_value FROM appbuilder_session_messages;")  
for row in cursor.fetchall():  
    print(json.loads(row[0]))

{'content': '你好', 'name': 'msg', 'mtype': 'str', 'id': '90c8d150-c7b9-44a4-ac77-50dd61ee329a'}
{'content': '你好，我是百度研发的知识增强大语言模型，中文名是文心一言，英文名是ERNIE Bot。我能够与人对话互动，回答问题，协助创作，高效便捷地帮助人们获取信息、知识和灵感。', 'name': 'msg', 'mtype': 'dict', 'id': '0fc6c5f7-23bf-4f25-8555-69b4155908b7', 'extra': {}, 'token_usage': {'prompt_tokens': 2, 'completion_tokens': 42, 'total_tokens': 44}}
{'content': '请介绍一下你自己', 'name': 'msg', 'mtype': 'dict', 'id': '4f5cf6d1-976e-456e-a632-93c4b2550523', 'extra': {'search_db': [{'content': '带机器人回复：请你扮演一个智能搜索改写补全机器人，请根据User的搜索历史以及对应的搜索结果，对最后一句话先进行主语继承改写，然后进行上下文信息补全，注意：不要改变原文的意思，答案要尽可能简洁，不要直接回答该问题，不要输出多于的内容。\\n\\n例子：\\n搜索历史：\\nUser：今天上午你干嘛了\\nAssistant：去打篮球啦\\nUser：好玩吗？\\n答案：\\n打篮球好玩吗？', 'dataset_id': '1f777fa3-26db-4237-98d5-075abc07a84f', 'dataset_name': '多轮改写Prompt', 'document_id': '92e0f30d-1f30-46f0-8377-59ab0fb6eb93', 'document_name': '多轮改写prompt_带机器人回复.txt', 'id': '61657591-f610-44f0-84ec-67cedcb66447', 'mock_id': '1', 'position': 0, 'score': 0.511053, 'sentences': [{'co

以id查询相关的agent_runtime信息

In [87]:
cursor.execute("SELECT * FROM appbuilder_session_messages WHERE session_id = 'b2c9d058-4475-4258-ad90-4334f3d024d5';")  
for tuple in cursor.fetchall():
    for message in tuple:
        try: 
            message = json.loads(message)
            print(message)
        except:
            print(message)

16010b88-d766-4524-81ee-37f96ceadb4d
b2c9d058-4475-4258-ad90-4334f3d024d5
0bdafb3d-f7e3-4187-bc1d-63cf51fbda29
query
{'content': '你好', 'name': 'msg', 'mtype': 'str', 'id': '90c8d150-c7b9-44a4-ac77-50dd61ee329a'}
2024-07-30 15:01:10.949475
2024-07-30 15:01:10.949485
0
4bd85fd0-2e5c-4de1-a47e-ea122f7e928c
b2c9d058-4475-4258-ad90-4334f3d024d5
0bdafb3d-f7e3-4187-bc1d-63cf51fbda29
answer
{'content': '你好，我是百度研发的知识增强大语言模型，中文名是文心一言，英文名是ERNIE Bot。我能够与人对话互动，回答问题，协助创作，高效便捷地帮助人们获取信息、知识和灵感。', 'name': 'msg', 'mtype': 'dict', 'id': '0fc6c5f7-23bf-4f25-8555-69b4155908b7', 'extra': {}, 'token_usage': {'prompt_tokens': 2, 'completion_tokens': 42, 'total_tokens': 44}}
2024-07-30 15:01:10.955875
2024-07-30 15:01:10.955884
0
d6e433fc-ef4d-4ade-9a63-fec600e95481
b2c9d058-4475-4258-ad90-4334f3d024d5
6300b76b-2307-4ed3-9f6d-e61dfc621ff8
query
{'content': '请介绍一下你自己', 'name': 'msg', 'mtype': 'dict', 'id': '4f5cf6d1-976e-456e-a632-93c4b2550523', 'extra': {'search_db': [{'content': '带机器人回复：请你扮演一个智能搜索改写补全机器人，请根据Us